In [3]:
import eland as ed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from elasticsearch import Elasticsearch


In [6]:
ed_flights = ed.read_es('localhost', 'kibana_sample_data_flights')

In [8]:
ed_flights.describe()

,AvgTicketPrice,DistanceKilometers,DistanceMiles,FlightDelayMin,FlightTimeMin,dayOfWeek
count,13059.000000,13059.000000,13059.000000,13059.000000,13059.000000,13059.000000
mean,628.253689,7092.142457,4406.853010,47.335171,511.127842,2.835975
std,266.386661,4578.263193,2844.800855,96.743006,334.741135,1.939365
min,100.020531,0.000000,0.000000,0.000000,0.000000,0.000000
25%,410.008918,2470.545974,1535.126118,0.000000,251.944994,1.000000
50%,640.387285,7612.072403,4729.922470,0.000000,503.148975,3.000000
75%,842.233478,9735.660463,6049.459005,11.080645,720.571142,4.308917
max,1199.729004,19881.482422,12353.780273,360.000000,1902.901978,6.000000


In [10]:
brainwave_df = pd.read_csv('~/Downloads/emotions.csv', index_col=False)

In [12]:
brainwave_df.describe()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b
count,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,...,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000
mean,15.256914,27.012462,-104.975629,13.605898,24.150483,0.025378,0.052282,0.301655,0.036793,0.083567,...,-22.938971,104.946111,-51.973647,-51.973647,104.946111,-6.934144,95.104886,-49.061255,-49.061255,95.104886
std,15.284621,9.265141,206.271960,16.874676,14.187340,17.981796,8.509174,68.098894,17.010031,18.935378,...,298.034311,212.532721,112.160233,112.160233,212.532721,281.040552,203.194976,106.486317,106.486317,203.194976
min,-61.300000,-114.000000,-970.000000,-137.000000,-217.000000,-218.000000,-255.000000,-1360.000000,-203.000000,-553.000000,...,-1180.000000,-921.000000,-504.000000,-504.000000,-921.000000,-1160.000000,-1010.000000,-521.000000,-521.000000,-1010.000000
25%,6.577500,26.075000,-195.000000,4.857500,23.600000,-3.105000,-1.340000,-4.002500,-2.905000,-2.622500,...,-106.500000,-8.365000,-92.900000,-92.900000,-8.365000,-102.500000,-8.837500,-87.150000,-87.150000,-8.837500
50%,14.100000,30.000000,14.950000,15.400000,25.200000,-0.044600,0.132000,0.957500,-0.099750,0.146500,...,83.850000,12.150000,-21.800000,-21.800000,12.150000,89.700000,13.400000,-24.100000,-24.100000,13.400000
75%,27.700000,31.400000,29.600000,26.500000,26.800000,2.920000,1.540000,6.735000,2.535000,2.870000,...,154.000000,177.000000,12.025000,12.025000,177.000000,153.000000,149.250000,10.925000,10.925000,149.250000
max,304.000000,42.300000,661.000000,206.000000,213.000000,402.000000,257.000000,1150.000000,349.000000,444.000000,...,1070.000000,843.000000,1490.000000,1490.000000,843.000000,1180.000000,888.000000,1670.000000,1670.000000,888.000000


In [14]:
label_df = brainwave_df['label']
brainwave_df.drop('label', axis = 1, inplace=True)
brainwave_df.head()

KeyError: 'label'

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split

pl_random_forest = Pipeline(steps=[('random_forest', RandomForestClassifier())])
scores = cross_val_score(pl_random_forest, brainwave_df, label_df, cv=10,scoring='accuracy')
print('Accuracy for RandomForest : ', scores.mean())

pl_random_forest

Accuracy for RandomForest :  0.9760732025882837


Pipeline(memory=None,
     steps=[('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [154]:
rfClassifier = pl_random_forest.steps[0][1]
fitted = rfClassifier.fit(brainwave_df, label_df)
rfClassifier.classes_

array(['NEGATIVE', 'NEUTRAL', 'POSITIVE'], dtype=object)

In [173]:
from typing import List
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.validation import check_is_fitted

def translate_tree_classifier(tree: DecisionTreeClassifier,
                              feature_names: List[str], 
                              classification_labels: List[str]=None) -> dict:
    """ Transform a DecisionTreeClassifier into a format accepted by Elastic ML inference
    
    This simply returns a dict that can then be serialized into JSON for transport
    
    Parameters
    ----------
    
    tree : DecisionTreeClassifier
           The trained/fitted decision tree to transform

    feature_names : array-like
                    The feature names that this decision tree cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc

    classification_labels : array-like, default=None
                            The labels to assign the classification results. 
                            
                            If no labels are supplied, inference returns the numerical value
                            that the tree returns.
                            
    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    """
    check_is_fitted(tree,["tree_", "classes_"])
    tree_state = tree.tree_.__getstate__()
    nodes = list()
    for i in range(len(tree_state["nodes"])):
        nodes.append(translate_node(i,tree_state["nodes"][i],tree_state["values"][i]))
    tree_dict = dict()
    tree_dict['tree'] = {
        "target_type": "classification",
        "feature_names": feature_names if feature_names is not None else [], #what to do???
        "tree_structure": nodes,
        "classification_labels": classification_labels if classification_labels is not None else list(tree.classes_),
    }
    return tree_dict 
    
def translate_node(idx,node,value) -> dict:
    node_dict = dict()
    node_dict["node_index"] = idx
    # Scikit learn tree nodes are always lte comparisons 
    # See https://github.com/scikit-learn/scikit-learn/blob/b194674c42d54b26137a456c510c5fdba1ba23e0/sklearn/tree/_tree.pyx#L812
    node_dict["decision_type"] = "lte"
    if node[0] == -1: # is leaf node
        # sets the value to the regression value stored in the tree value
        # Or to the classification value, which is the index of the largest value
        node_dict["leaf_value"] = int(np.argmax(value)) #TODO look into regression
    else:
        node_dict["left_child"] = int(node[0])
        node_dict["right_child"] = int(node[1])
        node_dict["split_feature"] = int(node[2])
        node_dict["threshold"] = float(node[3])
    return node_dict
    

In [183]:
from sklearn.ensemble.forest import ForestClassifier

def translate_classifier(classifier,
                         feature_names: List[str], 
                         classification_labels: List[str]=None, 
                         classification_weights: List[float]=None):
    if isinstance(classifier,DecisionTreeClassifier):
        return translate_tree_classifier(classifier,feature_names,classification_labels)
    elif isinstance(classifier,ForestClassifier):
        return translate_ensemble_classifier(classifier,feature_names,classification_labels,classification_weights)
    else:
        raise ValueError('Unexpected classifier model type ' + type(classifier))
        

def translate_ensemble_classifier(ensemble_classifier,
                                  feature_names: List[str], 
                                  classification_labels: List[str]=None, 
                                  classification_weights: List[float]=None):
    """ Transforms an ensemble classifier into the appropriate format for Elast ML utilization
    NOTE: the current ensemble models supported [ForestClassifier]
    Parameters
    ----------
    
    ensemble_classifier : ensemble classifier 
                          The trained/fitted ensemble classifier to transform 

    feature_names : array-like
                    The feature names that this model cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc

    classification_labels : array-like, default=None
                            The labels to assign the classification results. 
                            
                            If no labels are supplied, the classifier is interrogated
                            for the appropriate labels.
    classification_weights : array-like, default=None

                             The weights applied for specific classes on inference
                            
    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    
    """
    check_is_fitted(ensemble_classifier,["estimators_", "classes_"])
    models = ensemble_classifier.estimators_
    output_aggregator = dict()
    # sklearn effectively does a weighted vote for ForestClassifiers
    if isinstance(ensemble_classifier,ForestClassifier):
        output_aggregator["weighted_mode"] = { 
            "weights": [1.0] * len(models), 
            "max_class_value": len(ensemble_classifier.classes_) - 1 }
    else:
        raise ValueError('Unexpected model type ' + type(ensemble_classifier))
    model = dict()
    model['ensemble'] = {
        'target_type': 'classification',
        'feature_names': feature_names,
        'classification_labels': classification_labels if classification_labels is not None else list(ensemble_classifier.classes_),
        'trained_models': [translate_classifier(m,feature_names,classification_labels,classification_weights) for m in models],
        'aggregate_output': output_aggregator,
    }
    if classification_weights is not None:
        model['ensemble']['classification_weights'] = classification_weights
    return model
    
    

In [157]:
translate_classifier(fitted,[])

FileNotFoundError: [Errno 2] File b'iris.data.csv' does not exist: b'iris.data.csv'

In [188]:
iris = pd.read_csv('~/Downloads/iris.data', header = None)
#Renaming the columns
iris.columns = ['sepal_length', 'sepal_width','petal_length','petal_width','species']
print('Shape of the dataset: ' + str(iris.shape))
print(iris.head(100))
print(iris.taill(100))

iris_labels = iris['species']
iris.drop('species', axis = 1, inplace=True)

irisClassifer = RandomForestClassifier()
irisFitted = irisClassifer.fit(iris, iris_labels)
scores = cross_val_score(irisClassifer, iris, iris_labels, cv=10,scoring='accuracy')


Shape of the dataset: (150, 5)
    sepal_length  sepal_width  petal_length  petal_width          species
0            5.1          3.5           1.4          0.2      Iris-setosa
1            4.9          3.0           1.4          0.2      Iris-setosa
2            4.7          3.2           1.3          0.2      Iris-setosa
3            4.6          3.1           1.5          0.2      Iris-setosa
4            5.0          3.6           1.4          0.2      Iris-setosa
..           ...          ...           ...          ...              ...
95           5.7          3.0           4.2          1.2  Iris-versicolor
96           5.7          2.9           4.2          1.3  Iris-versicolor
97           6.2          2.9           4.3          1.3  Iris-versicolor
98           5.1          2.5           3.0          1.1  Iris-versicolor
99           5.7          2.8           4.1          1.3  Iris-versicolor

[100 rows x 5 columns]
     sepal_length  sepal_width  petal_length  petal_width

In [184]:
es_ensemble=translate_classifier(fitted,['sepal_length', 'sepal_width','petal_length','petal_width'])

import json

jsonString = json.dumps({'trained_model':es_ensemble})


import gzip
import base64
compressed_value = base64.b64encode(gzip.compress(bytes(jsonString,'utf-8')))
compressed_value

b'H4sIAMHqMV4C/+1b7Y6bOhB9lVV+b5HHX9h9g/sMVYVo4iRIbBIBaXtV9d0vJr23uzGKJwwsu73kx0qwgI+xZ+bMmeHHqqny4uA22dNx48rVx4cfK3eo3dOX0nUHTV7tXJM1f5/88Wpd5nVdbIt13hTHw+rxYbV1eXOuXHbIn1zdXvJpVbtTXmalO+yavb/icvyt2FwOT6558e/L8eXfnx+vh8jK/IsrLw/+qyrqD7VrjnXub+wOv7qqLtbH8lj9PlVUu+LQ3t897sX8uue0k6rcXLPzQ2d1U53X/rkXOIcWWlYcNu57e8zaizZuXdR+8v8CKxvnH1a6bZOt90W5aU9Ce6IqdvvfZ7iHcyqLJvsFuz0n/Jj7ytX7Y3cNSwzrfgCWM8vtz8eHKwRwE0G+zb7m5dmfZOG9HIteBOiB9cDnV/BlYpRHLwxIbVOehhAEFoIMICgUglSFg0rsO4PwXoUFrAPAKWLFIVG2+zGpBVfayBCCJsBPsfBNAN/2wGdX8FVipUdv2s2qFYieFTcE9BZ7L+8xFLythsYKHDF5fb12PSCAMgO0vUKPwUrU7uszFxCEJQOJn/HNgMKS9n2D/8MT9vlnN9ISGt5gaNCojaa7yMCsEjqVAvQMoSHqqqaLE7cNXS8mc6/JGFRwkrEVBywCHiAQiD3HEdGdE6yWYDIp4gWSTWb70kTjfgOCFygvfkOm2oi0z20ogtFqgtGmhHEN9q1ZXFwf4Oss2vZYiAFQez/u8iMc7fbmB7TxQmi9IHD0iD3ffcrcTdIiZElQ6CHa9iA0Ps4RS6gS6LaRBqME46noAYHOjyBMkDgM56gaPW4ajstw7L5nWHQ+BWFCBRaXEHbv3Cremgw3sm/lDWXbWMKG5YxyMyUZ4nwhSWPkFUMoylyak0QxBBtjCASWZMdRvQgsyYzD00aWsQbwtOlUrNuOYzr9KUKSxDhEbXwdC4YwNaDQHE6JV7NxJNBjcURKjgKLOnB34NModU